In [2]:
from racepi_database_handler import *
from racepi_can_decoder import *
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from datetime import datetime
from collections import defaultdict
from ipywidgets import interact
import numpy as np
import math

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()   
def get_session_data(db, session_id):
    data = defaultdict(list)
    data['gps'] = db.query(GPSData).filter(GPSData.session_id == session_id).order_by(GPSData.timestamp).all()
    data['can'] = db.query(CANData).filter(CANData.session_id == session_id).order_by(CANData.timestamp).all()
    data['imu'] = db.query(IMUData).filter(IMUData.session_id == session_id).order_by(IMUData.timestamp).all()
    return data

def load_dbfile(filename):
    engine = create_engine('sqlite:///' + filename)
    Base.metadata.bind = engine
    sm = sessionmaker()
    sm.bind = engine
    s = sm()
    return s

s = load_dbfile("/home/donour/test.db")
si = s.query(SessionInfo).filter(SessionInfo.max_speed >15).all()
for session in si:
    session_date = datetime.fromtimestamp(session.start_time_utc)
    print(session_date, session.session_id, session.duration)

Loading BokehJS ...

2017-04-02 10:16:53.374481 0b4f4ed417af11e79bbbb827eb8021e9 87.49850845336914
2017-04-02 10:36:29.268030 c835a6fe17b111e79bbbb827eb8021e9 76.59503507614136
2017-04-02 10:56:02.560940 838ef27817b411e79b6ab827eb8021e9 78.40034413337708
2017-04-02 11:03:57.974179 9ee0854a17b511e79b6ab827eb8021e9 73.50186252593994
2017-04-02 11:14:08.878915 0b0e938217b711e79b6ab827eb8021e9 80.60055637359619
2017-04-02 11:20:02.794504 ddf1c8b417b711e7b2dcb827eb8021e9 57.0844988822937
2017-04-02 11:28:00.775138 faddb75217b811e7a360b827eb8021e9 59.69824981689453
2017-04-02 11:34:27.367086 e1551bc617b911e7a176b827eb8021e9 84.99559879302979
2017-04-02 11:43:40.680222 2b1aedfc17bb11e7a176b827eb8021e9 62.63226652145386
2017-04-02 11:59:57.855384 7184bf5a17bd11e7a176b827eb8021e9 78.69126272201538
2017-04-02 12:17:44.133480 ed12c4f817bf11e79fe5b827eb8021e9 82.50079154968262
2017-04-02 12:37:39.681180 b5abb0f817c211e79fe5b827eb8021e9 79.39723086357117
2017-04-02 12:53:28.384996 eb2e610617c411e79fe5b827eb8021e9 81.20

In [3]:
data = get_session_data(s,"d16c2b5c44a811e7bc09b827eb8021e9")

x = [v.timestamp for v in data.get("imu")]
y = [v.x_accel for v in data.get("imu")]
y2 = [v.z_gyro for v in data.get("imu")]

p = figure(title="Accel vs. Gyro", plot_height=300, plot_width=900)
r = p.line(x, y, color="#2222aa", line_width=1, legend="Lateral Accel (g)")
r = p.line(x, y2, color="#ff0000", line_width=1, legend="Yaw (rad/s)")
show(p, notebook_handle=True)

x = [v.timestamp for v in data.get("gps")]
y = [v.speed for v in data.get("gps")]
p = figure(title="Speed", plot_height=300, plot_width=900)
r = p.line(x, y, color="#2222aa", line_width=1, legend="Lateral Accel (g)")
show(p, notebook_handle=True)

In [4]:
# caclulate understeer/oversteer
from math import cos, fabs, pi

steering_ratio = 13.0
def msg_to_steering_angle(msg):
    return focus_rs_steering_angle_converter.convert_frame(CanFrame("999", msg)) / steering_ratio *\
     (focus_rs_steering_direction_converter.convert_frame(CanFrame("999", msg))*2-1)

def oversteer_coefficient(steering_angle, wheelbase, velocity, yaw_rate):
    if fabs(steering_angle) > 1e-15:
        requested_turn_radius = (wheelbase) / cos((pi/2)-steering_angle)
        requested_yaw = velocity / requested_turn_radius
    else:
        requested_yaw = 0.0    
        
    return requested_yaw/yaw_rate / 1000.0

steering_samples = [x for x in data.get('can') if x.arbitration_id==0x10]

x = []
y = []
gps_iter = iter(data.get('gps'))
imu_iter = iter(data.get('imu'))
last_gps = next(gps_iter, None)
last_imu = next(imu_iter, None)
for v in steering_samples:
    if last_gps and last_imu:
        x.append(v.timestamp)
        y.append(oversteer_coefficient(msg_to_steering_angle(v.msg), 2.64922, last_gps.speed, last_imu.z_gyro))
    while last_gps and last_gps.timestamp < v.timestamp:
        last_gps = next(gps_iter, None)
    while last_imu and last_imu.timestamp < v.timestamp:
        last_imu = next(imu_iter, None)
        

x2 = [v.timestamp for v in steering_samples]
y2 = [msg_to_steering_angle(x.msg) for x in steering_samples]
x3 = [v.timestamp for v in data.get("imu")]
y3 = [v.z_gyro for v in data.get("imu")]
   

p = figure(title="Oversteer/understeer", plot_height=300, plot_width=900)
r = p.line(x, y, color="#ff0000", line_width=1, legend="understeer")
r = p.line(x2, y2, color="#2222aa", line_width=1, legend="Steering")
r = p.line(x3, y3, color="#00ff00", line_width=1, legend="Yaw (rad/s)")
show(p, notebook_handle=True)

In [9]:
brake_samples = s.query(CANData).filter(CANData.session_id == 'd16c2b5c44a811e7bc09b827eb8021e9').filter(CANData.arbitration_id == 0x213).order_by(CANData.timestamp).all()

x = [v.timestamp for v in brake_samples]
y = [focus_rs_brake_pressure_converter.convert_frame(CanFrame("999", v.msg)) for v in brake_samples]

p = figure(title="Brake Pressure", plot_height=300, plot_width=900)
r = p.line(x, y, color="#ff0000", line_width=1, legend="brake")
show(p, notebook_handle=True)

In [1]:
from racepi_can_decoder import *